In [2]:
!pip3 install -q transformers accelerate sentence-transformers langchain langchain-community pypdf faiss-cpu gradio safetensors requests==2.32.4 bitsandbytes

In [3]:
# mount drive in colab
from google.colab import drive
drive.mount('/content/drive')

# assert torch cuda
import torch
torch.cuda.is_available()

Mounted at /content/drive


True

In [4]:
import os
from langchain_community.document_loaders import PyPDFLoader

# Path to your PDFs
pdf_dir = "/content/drive/MyDrive/Colab Notebooks/data/pdfs/spei_mundus"
docs = []

for file in os.listdir(pdf_dir):
    if file.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(pdf_dir, file))
        docs.extend(loader.load())

print(f"Loaded {len(docs)} pages from {len(os.listdir(pdf_dir))} PDFs")
docs[0]

Loaded 28 pages from 6 PDFs


Document(metadata={'producer': 'Skia/PDF m80', 'creator': 'Chromium', 'creationdate': '2025-07-17T16:00:38+00:00', 'moddate': '2025-07-17T16:00:38+00:00', 'source': '/content/drive/MyDrive/Colab Notebooks/data/pdfs/spei_mundus/john.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content="john.md 2025-07-17\n1 / 5\nJOHN\nCONCEPT\nAncien rangers de Nouvelle Moscou a la retraite.\nCATEGORIE D'AGE\nAncien.\nexperience totale experience depensee experience restante\n65 65 0")

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer

# Split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
splits = splitter.split_documents(docs)

# Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(splits, embedding_model) # improve later : 2 steps : approximative + flat
db

/tmp/ipython-input-928964704.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "DeepSeek-R1-Distill-Qwen-1.5B"
save_path = f"/content/drive/MyDrive/Colab Notebooks/models/{model_name}"
model_path = f"deepseek-ai/{model_name}"
tokenizer, model = None, None

try:
  tokenizer = AutoTokenizer.from_pretrained(save_path)
  model = AutoModelForCausalLM.from_pretrained(
      save_path,
      device_map="auto",
      torch_dtype=torch.float16,
      low_cpu_mem_usage=True,
  )
except Exception as e:
  tokenizer = AutoTokenizer.from_pretrained(model_path)
  model = AutoModelForCausalLM.from_pretrained(
      model_path,
      device_map="auto",
      torch_dtype=torch.float16,
      low_cpu_mem_usage=True,
  )

  tokenizer.save_pretrained(save_path)
  model.save_pretrained(save_path)
  model = torch.compile(model)

tokenizer, model

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

(LlamaTokenizerFast(name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', vocab_size=151643, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
 	151643: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151644: AddedToken("<｜User｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
 	151645: AddedToken("<｜Assistant｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
 	151646: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151647: AddedToken("<|EOT|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
 	151648: AddedToken("<think>", rstri

In [7]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

system_prompt = "You are an information retrieving agent. Always give factuals and structured answers."

# LLM pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.2,
    do_sample=False, # deterministic approach: argmax
)
llm = HuggingFacePipeline(pipeline=pipe)

prompt = "Who is John? What relation does he has with Selene?"
final_prompt = f"SYSTEM:{system_prompt}\nUser:{prompt}\nAgent:"

torch.cuda.empty_cache()
with torch.no_grad():
  output_raw = pipe(final_prompt)

output_raw

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The model 'OptimizedModule' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone

[{'generated_text': "SYSTEM:You are an information retrieving agent. Always give factuals and structured answers.\nUser:Who is John? What relation does he has with Selene?\nAgent: John is a historical figure, and Selene is a historical figure. They are related through their roles in a historical event, specifically the Battle of the Somme. John was a general who played a significant role in the battle, while Selene was a general as well. They were both involved in the battle, but their exact roles and contributions differ. John's role was more tactical and strategic, while Selene's was more tactical and tactical. Together, they were able to achieve the objectives of the battle and protect the French forces from the German forces."}]

In [8]:
# Retrieval QA
retriever = db.as_retriever(search_kwargs={"k": 5})

qa_chain_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # simplest: stuff all retrieved docs into prompt
)

torch.cuda.empty_cache()
with torch.no_grad():
  output_rag_stuff = qa_chain_stuff.run(final_prompt)

output_rag_stuff

/tmp/ipython-input-378591350.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_rag_stuff = qa_chain_stuff.run(final_prompt)
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nLangage machine [competence]\nJe les comprends et ressens leurs pensées, sans jamais avoir eu à interagir avec eux. Quelle que soit\nl\'origine de leur nature logique, les systèmes automatiques de ces entités semblent me considérer comme\nl\'un des leurs...\nCommuniquer avec tout systeme ALIEN et toute IA. Peut ouvrir la plupart des portes\ntechnologiques. N\'est jamais considere comme une menace par certains systemes automatiques.\nMaitrise latente [competence]\n\nselene.md 2025-07-17\n1 / 5\nSELENE\nCONCEPT\nAncien rangers de Nouvelle Moscou a la retraite.\nCATEGORIE D\'AGE\nVeteran.\nexperience totale experience depensee experience restante\n35 35 0\n\nnikita.md 2025-07-17\n2 / 5\nCaracteristiques & Competences\nCHARISME CONNAISSANCES CORPS PERCEPTION\n3 5 2 3\nempathie artisanat(electronique) experience de la rue\n

In [10]:
# Retrieval QA
retriever = db.as_retriever(search_kwargs={"k": 1})
qa_chain_mr = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="map_reduce",  # parallel calls for each trunks then regroup
    chain_type_kwargs={"token_max": 4096}
)

torch.cuda.empty_cache()
with torch.no_grad():
  output_rag_mr = qa_chain_mr.run(final_prompt)

output_rag_mr

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2738 > 1024). Running this sequence through the model will result in indexing errors
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'Given the following extracted parts of a long document and a question, create a final answer.\nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\n\nContent: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and thi

In [ ]:
# Retrieval QA
retriever = db.as_retriever(search_kwargs={"k": 5})
qa_chain_refine = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="refine",  # linear iterative: chunk after chunk
)

torch.cuda.empty_cache()
with torch.no_grad():
  output_rag_refine = qa_chain_refine.run(final_prompt)

output_rag_refine

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
print(output_raw)
print()

print(output_rag_stuff.split('</think>')[-1])
print()

print(output_rag_mr.split('</think>')[-1])
print()

print(output_rag_refine.split('</think>')[-1])
print()

In [ ]:
import gradio as gr

def ask_agent(question):
    return qa_chain.run(question)

demo = gr.Interface(
    fn=ask_agent,
    inputs=gr.Textbox(label="Ask about your PDFs"),
    outputs=gr.Textbox(label="Answer")
)

demo.launch(share=True)
